1. Load Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.mixture import GaussianMixture
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
from joblib import dump

In [8]:
# import the dataset
beer_df = pd.read_csv('../data/raw/beer_reviews.zip')

2. Explore Data

In [7]:
pd.set_option('display.max_columns', None)
beer_df.head(10)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [4]:
beer_df.tail(10)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1586604,14359,The Defiant Brewing Company,1288890206,4.0,4.5,4.5,njmoons,Pumpkin Ale,3.5,3.5,The Horseman's Ale,5.2,33061
1586605,14359,The Defiant Brewing Company,1163291143,5.0,5.0,5.0,NyackNicky,Pumpkin Ale,5.0,5.0,The Horseman's Ale,5.2,33061
1586606,14359,The Defiant Brewing Company,1162871808,5.0,4.5,4.0,blitheringidiot,Pumpkin Ale,5.0,5.0,The Horseman's Ale,5.2,33061
1586607,14359,The Defiant Brewing Company,1162865640,5.0,5.0,4.5,PopeDX,Pumpkin Ale,5.0,4.5,The Horseman's Ale,5.2,33061
1586608,14359,The Defiant Brewing Company,1162685856,3.5,4.0,4.0,treehugger02010,Pumpkin Ale,3.5,3.0,The Horseman's Ale,5.2,33061
1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061
1586612,14359,The Defiant Brewing Company,1160023044,4.0,4.5,4.5,dherling,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061
1586613,14359,The Defiant Brewing Company,1160005319,5.0,4.5,4.5,cbl2,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061


In [5]:
# dimensions of df
beer_df.shape

(1586614, 13)

In [6]:
# summary of df
beer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [7]:
# general description of data
beer_df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


3. Prepare Data

In [8]:
# create a copy of df for data preparation
beer_df_cleaned = beer_df.copy()

In [9]:
# Drop the unnecessary columns
beer_df_cleaned.drop(['brewery_id','review_time', 'review_overall', 'review_profilename', 'beer_style', 'beer_beerid'], axis=1, inplace=True)

In [10]:
beer_df_cleaned.head()

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_name,beer_abv
0,Vecchio Birraio,2.0,2.5,1.5,1.5,Sausa Weizen,5.0
1,Vecchio Birraio,2.5,3.0,3.0,3.0,Red Moon,6.2
2,Vecchio Birraio,2.5,3.0,3.0,3.0,Black Horse Black Beer,6.5
3,Vecchio Birraio,3.0,3.5,2.5,3.0,Sausa Pils,5.0
4,Caldera Brewing Company,4.5,4.0,4.0,4.5,Cauldron DIPA,7.7


In [11]:
beer_df_cleaned.isna().sum()

brewery_name            15
review_aroma             0
review_appearance        0
review_palate            0
review_taste             0
beer_name                0
beer_abv             67785
dtype: int64

In [12]:
beer_df_cleaned[beer_df_cleaned['beer_abv'].isna()]

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_name,beer_abv
273,Caldera Brewing Company,3.0,3.0,4.0,3.0,Cauldron Espresso Stout,NaN
430,Moon River Brewing Company,4.0,4.5,3.5,3.5,The Highland Stagger,NaN
603,Moon River Brewing Company,3.5,4.0,3.5,3.5,The Highland Stagger,NaN
733,Caldera Brewing Company,4.0,4.0,4.0,4.0,Alpha Beta,NaN
798,Caldera Brewing Company,4.5,4.0,4.0,4.5,Imperial Stout,NaN
...,...,...,...,...,...,...,...
1586568,The Defiant Brewing Company,3.5,4.0,4.0,4.0,Bock,NaN
1586587,The Defiant Brewing Company,4.5,4.0,4.5,4.0,Maibock,NaN
1586596,The Defiant Brewing Company,3.0,5.0,4.0,3.5,Resolution #2,NaN
1586597,The Defiant Brewing Company,4.5,4.0,4.0,4.0,Resolution #2,NaN


In [13]:
beer_df_cleaned.dropna(subset=['beer_abv'], inplace=True)

In [14]:
beer_df_cleaned[beer_df_cleaned['brewery_name'].isna()]

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_name,beer_abv
651565,NaN,2.5,2.5,1.5,1.5,Engel Tyrolian Bräu WRONG BREWERY SEE SCHWABIS...,5.0
659293,NaN,4.5,3.5,4.0,3.5,Engel Bock Dunkel WRONG BREWERY SEE CRAILSHEIMER,7.2
659299,NaN,3.0,3.0,4.0,4.0,Engel Gold WRONG BREWERY SEE CRAILSHEIMER,5.4
659300,NaN,4.0,3.5,3.5,3.0,Engel Landbier WRONG BREWERY SEE CRAILSHEIMER,4.8
659301,NaN,4.0,4.0,4.0,3.5,Engel Keller Hell WRONG BREWERY SEE CRAILSHEIMER,5.4
659302,NaN,4.0,3.0,3.0,3.5,Engel Aloisius - WRONG BREWERY SEE CRAILSHEIMER,5.9
659303,NaN,3.0,3.0,2.0,3.0,Engel Keller Dunkel WRONG BREWERY SEE CRAILSH...,5.3
659304,NaN,4.0,4.0,4.0,4.5,Engel Keller Dunkel WRONG BREWERY SEE CRAILSH...,5.3
659305,NaN,3.5,4.0,4.0,4.0,Engel Keller Dunkel WRONG BREWERY SEE CRAILSH...,5.3
1391043,NaN,3.5,3.5,4.0,3.5,Hard Hat American Beer,3.8


In [15]:
# Replace null values of brewery name to 'Unknown'
beer_df_cleaned['brewery_name'] = beer_df_cleaned['brewery_name'].fillna('Others')

In [16]:
beer_df_cleaned.isna().sum()

brewery_name         0
review_aroma         0
review_appearance    0
review_palate        0
review_taste         0
beer_name            0
beer_abv             0
dtype: int64

In [17]:
beer_df_cleaned['brewery_name'].value_counts()

brewery_name
Boston Beer Company (Samuel Adams)          38812
Dogfish Head Brewery                        33800
Stone Brewing Co.                           33022
Sierra Nevada Brewing Co.                   28637
Bell's Brewery, Inc.                        24975
                                            ...  
Statale Nove                                    1
Egyptian International Beverages Company        1
Tofino Brewing Company                          1
Brauerei Allersheim GmbH                        1
Kennedy School (McMenamins)                     1
Name: count, Length: 5156, dtype: int64

In [18]:
# Get the names of breweries with counts less than a threshold
brewery_name_counts = beer_df_cleaned['brewery_name'].value_counts()
breweries_to_filter = brewery_name_counts[brewery_name_counts < 10].index

# Replace 'brewery_name' values for the filtered breweries with 'unknown/others'
beer_df_cleaned.loc[beer_df_cleaned['brewery_name'].isin(breweries_to_filter), 'brewery_name'] = 'Others'

In [19]:
beer_df_cleaned['brewery_name'].value_counts()

brewery_name
Boston Beer Company (Samuel Adams)    38812
Dogfish Head Brewery                  33800
Stone Brewing Co.                     33022
Sierra Nevada Brewing Co.             28637
Bell's Brewery, Inc.                  24975
                                      ...  
Oakbank Brewing                          10
Bières 23, Brasserie Artisanale          10
Prescott Brewing Company                 10
Skagway Brewing Company                  10
Paulista Brewery                         10
Name: count, Length: 2846, dtype: int64

In [20]:
beer_df_cleaned.shape

(1518829, 7)

In [21]:
# Save cleaned dataset into the interim data folder
filename = '../data/interim/beer_reviews_cleaned.csv'
compression_options = dict(method='zip', archive_name=f'{filename}.csv')
beer_df_cleaned.to_csv(f'{filename}.zip', compression=compression_options,index=False)

In [22]:
# Split the data into training and testing sets with 80-20 ratio
X = beer_df_cleaned.drop(['beer_name'], axis=1)
y = beer_df_cleaned['beer_name']

# Stratify based on response
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Check the shapes
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1215063, 6)
Shape of X_test: (303766, 6)
Shape of y_train: (1215063,)
Shape of y_test: (303766,)


In [24]:
# Create a list of numerical feautures
num_features = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste','beer_abv']

In [25]:
# Create a list of categorical features
cat_features = ['brewery_name']

4. Build Pipeline

In [26]:
# Create a Pipeline called num_transformer with one step that contains StandardScaler
num_transformer = Pipeline(
     steps=[
        ('scaler', StandardScaler())
    ]
)

In [27]:
# Create a Pipeline called cat_transformer with one step that contains OneHotEncoder
cat_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False, drop='first'))
    ]
)

In [28]:
# Create a ColumnTransformer called preprocessor with 2 steps containing num_transformer and cat_transformer that will be applied respectively to num_features and cat_features
preprocessor = ColumnTransformer(
    transformers=[
        ('num_features', num_transformer, num_features),
        ('cat_features', cat_transformer, cat_features)
    ]
)

In [29]:
knn_classifier = KNeighborsClassifier(n_jobs=-1)

In [30]:
# Create a Pipeline called gm_pipe that contains 2 steps preprocessor and another that instantiate a GaussianMixture 
knn_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('clustering', knn_classifier)
    ]
)

In [31]:
# Fit model
knn_pipeline.fit(X_train, y_train)

/opt/homebrew/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_features',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['review_aroma',
                                                   'review_appearance',
                                                   'review_palate',
                                                   'review_taste',
                                                   'beer_abv']),
                                                 ('cat_features',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['brewery_name'])])),
                ('clustering', KNeighborsClassifier(n_jobs=-1))])

In [32]:
# Prediction
y_pred = knn_pipeline.predict(X_test)

In [33]:
# Evaluate 
accuracy = accuracy_score(y_test, y_pred)
print({"Accuracy": round(accuracy, 4)})

{'Accuracy': 0.5351}


In [34]:
import pickle

In [37]:
# Store the Brewery Names
brewery_names = np.sort(beer_df_cleaned['brewery_name'].unique())

with open('../models/brewery_names.pkl', 'wb') as file:
    pickle.dump(brewery_names, file)

In [41]:
# Save the model
dump(knn_pipeline, '../models/knn_pipeline_compression_7.joblib', compress=7)

['../models/knn_pipeline_compression_7.joblib']